In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import transformers 
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('../input/contradictory-my-dear-watson/train.csv')
test_df = pd.read_csv('../input/contradictory-my-dear-watson/test.csv')

In [ ]:
def init_TPU():
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        REPLICAS = strategy.num_replicas_in_sync
        print('connected to tpu')
        return strategy
    except ValueError:
        print('conneccct tpu failed')
        
        return tf.distribute.get_strategy()

In [ ]:
strategy = init_TPU()

In [ ]:
model_path = 'jplu/tf-xlm-roberta-base'
max_len = 80
batch_size = 16 * strategy.num_replicas_in_sync
AUTO = tf.data.experimental.AUTOTUNE
epochs = 20
n_steps = len(train_df) // batch_size

In [15]:
def build_model(strategy, transformer):
    with strategy.scope():
        transformer_encoder = TFAutoModel.from_pretrained(transformer)
        input_layer = Input(shape=(max_len,), dtype=tf.int32)
        sequence_output = transformer_encoder(input_layer)[0]
        cls_token = sequence_output[:, 0, :]
        output_layer = Dense(3, activation='softmax')(cls_token)
        
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy']
                     )
        return model

In [16]:
model = build_model(strategy, model_path)

In [17]:
train_df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [21]:
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
import seaborn as sns
import plotly.express as px
fig = px.bar(train_df, x=train_df['language'])
iplot(fig)

In [22]:
# 균등한 분포를
# 최대 count 영어

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [24]:
train_data = train_df[['premise', 'hypothesis']].values.tolist()

In [25]:
test_data = test_df[['premise', 'hypothesis']].values.tolist()

In [27]:
# encoding
train_encoded = tokenizer.batch_encode_plus(train_data,
                                            pad_to_max_length=True,
                                            max_length=max_len)


In [28]:
# encoding
test_encoded = tokenizer.batch_encode_plus(test_data,
                                            pad_to_max_length=True,
                                            max_length=max_len)


In [35]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_encoded['input_ids'],
    train_df.label.values,
    test_size=0.2
)

x_test = test_encoded['input_ids']

In [36]:
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
                        .repeat()
                        .shuffle(2048)
                        .batch(batch_size)
                        .prefetch(AUTO)
                )

In [37]:
valid_dataset = (
    tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)

In [38]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
)

In [40]:
model.fit(train_dataset, steps_per_epoch=n_steps,
          epochs=epochs,
         validation_data=valid_dataset)

Epoch 1/20
94/94 [==============================] - ETA: 0s - loss: 1.1333 - accuracy: 0.3282

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})'}), <class 'numpy.ndarray'>